In [11]:
import pandas as pd
from serpapi import GoogleSearch
import time
import re

In [12]:
# Load the Excel file
df = pd.read_excel("companylist.xlsx")

In [13]:
print(df)

      companyId                                            company
0             1  Bhabha Atomic Research Centre||BHABHA ATOMIC R...
1             2  Assam Power Generation Corporation Limited  Po...
2             3                Container Corporation of India Ltd.
3             4  Directorate Of Horticulture  Agriculture And C...
4             5  Directorate Of Agriculture  Agriculture And Co...
...         ...                                                ...
4995       4996  Karnataka Backward Classes Department Building...
4996       4997  Jai Yeshwant Co-operative Housing Society Limited
4997       4998  Andheri Sarabjit Co-operative Housing Society ...
4998       4999   Juhu Ajanta Co-operative Housing Society Limited
4999       5000          Uttar Bhartiya Sehkari Awas Nigam Limited

[5000 rows x 2 columns]


In [14]:
# Only do the first 1000 entries for testing
sample_df = df.head(1000)

In [15]:
print(sample_df)

     companyId                                            company
0            1  Bhabha Atomic Research Centre||BHABHA ATOMIC R...
1            2  Assam Power Generation Corporation Limited  Po...
2            3                Container Corporation of India Ltd.
3            4  Directorate Of Horticulture  Agriculture And C...
4            5  Directorate Of Agriculture  Agriculture And Co...
..         ...                                                ...
995        996     Ministry of External Affairs||EOI ROME (ITALY)
996        997  Ministry of Coal  Coal India Limited COAL INDI...
997        998  CE Small Scale Irrigation (WC) Pune||SE Small ...
998        999  Soil and Water Conservation Department||Additi...
999       1000  District Treasury Office Patan Finance Departm...

[1000 rows x 2 columns]


In [16]:
# Your SerpAPI API Key
API_KEY = "f4ca380ca7e5c32f1aeb79445ddc44a28235719b325f96ac2beb7c1ed1839811"

In [17]:
# Define unwanted location endings
UNWANTED_LOCATIONS = {"india", "gujarat", "maharashtra", "delhi", "jammu", "bangalore", "mumbai", "ahmedabad"}

In [18]:
import requests

# Function to get best match from Serper.dev
def get_best_match(query):
    url = "https://google.serper.dev/search"
    headers = {
        "X-API-KEY": "6f08ea973f3cc1b8cf11887cc20b6677a9d3a21d",  # Replace with your real API key
        "Content-Type": "application/json"
    }
    data = {"q": query}

    response = requests.post(url, json=data, headers=headers)
    results = response.json()

    # Check if organic results are available
    if "organic" in results:
        for result in results["organic"][:3]:
            title = result.get("title", "").strip()
            if title and title.lower() not in UNWANTED_LOCATIONS:
                return title

    # If no relevant result found
    return "Not Found"


In [19]:
# Store the final results
results = []

In [20]:
for company in sample_df["company"]:
    # Clean query: remove repeated words like "Assam Assam"
    cleaned_query = re.sub(r"\b(\w+)(\s+\1\b)+", r"\1", str(company), flags=re.IGNORECASE)
    cleaned_query = re.sub(r"\s+", " ", cleaned_query).strip()

    best_name = get_best_match(cleaned_query)

    # Apply your custom logic only if "..." is present
    if "..." in best_name:
        # Step 1: Remove "..." itself
        best_name = best_name.replace("...", "").strip()

        # Step 2: Remove the last "&" only if it appeared before "..."
        last_amp = best_name.rfind("&")
        if last_amp != -1:
            best_name = best_name[:last_amp] + best_name[last_amp+1:]

        # Step 3: Smart check for removing text after last comma
        if "," in best_name:
            parts = best_name.rsplit(",", 1)
            after_comma = parts[1].strip().lower()
            word_count = len(after_comma.split())

            if word_count <= 2 or after_comma in UNWANTED_LOCATIONS:
                best_name = parts[0].strip()

    results.append({
        "Old Company Name": company,
        "New Company Name": best_name
    })

    print(f"Searching : {company} \n→ {best_name}")
    time.sleep(1.5)  # Respect API limits

Searching : Bhabha Atomic Research Centre||BHABHA ATOMIC RESEARCH CENTRE- Mumbai||Engineering Services Group - B 
→ Bhabha Atomic Research Centre ( BARC ), Department of Atomic
Searching : Assam Power Generation Corporation Limited  Power Department, Assam  Assam 
→ APGCL
Searching : Container Corporation of India Ltd. 
→ Container Corporation of India Ltd :: A Navratna Company
Searching : Directorate Of Horticulture  Agriculture And Co-operation Department Gujarat  Gujarat 
→ Director Of Horticulture
Searching : Directorate Of Agriculture  Agriculture And Co-operation Department Gujarat  Gujarat 
→ Agriculture, Farmers Welfare and Co-operation Department
Searching : Directorate Of Tourism Jammu Government Of Jammu &  Tourism Department Jammu And Kashmir  Jammu & Kashmir 
→ Directorate of Tourism Jammu, J&K
Searching : Publications Division Ministry Of Information And Broadcasting Ministry Of Information And Broadcast 
→ Publication Division,Ministry of information  broadcasting
Search

In [21]:
# Save results to DataFrame
result_df = pd.DataFrame(results)
print(result_df)
result_df.to_excel("/home/fenil/Desktop/NC1000_test.xlsx", index=False)
print("Saved to Desktop.")

                                      Old Company Name  \
0    Bhabha Atomic Research Centre||BHABHA ATOMIC R...   
1    Assam Power Generation Corporation Limited  Po...   
2                  Container Corporation of India Ltd.   
3    Directorate Of Horticulture  Agriculture And C...   
4    Directorate Of Agriculture  Agriculture And Co...   
..                                                 ...   
995     Ministry of External Affairs||EOI ROME (ITALY)   
996  Ministry of Coal  Coal India Limited COAL INDI...   
997  CE Small Scale Irrigation (WC) Pune||SE Small ...   
998  Soil and Water Conservation Department||Additi...   
999  District Treasury Office Patan Finance Departm...   

                                      New Company Name  
0    Bhabha Atomic Research Centre ( BARC ), Depart...  
1                                                APGCL  
2    Container Corporation of India Ltd :: A Navrat...  
3                             Director Of Horticulture  
4    Agriculture, 